In [123]:
# These settings automatically re-render the widget when the JavaScipt file has changed

%load_ext autoreload
%autoreload 2
%env ANYWIDGET_HMR=1

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
env: ANYWIDGET_HMR=1


In [163]:
from ipywidgets import Button, GridBox, Layout, ButtonStyle, IntSlider, Box, jslink, HTML
from nzshm_rupture_widget import rupture_map, MapWidget, SliderWidget, FullScreenWidget, HomeWidget, legend, slider, MapLayout
import json

In [158]:
with open('small_styled_crustal.geojson') as json_file:
    styled_crustal = json.load(json_file)
    json_file.close()

polygon = {"type":"FeatureCollection","features": [{
    "type": "Feature",
    "properties": {},
    "geometry": {
        "coordinates": [
            [
                [31.82, 9.27, 1],
                [31.82, 5.64, 1],
                [34.87, 5.64, 1],
                [34.87, 9.27, 1]
            ]
        ],
        "type": "Polygon"
    }
}]}

color_map = {0.001: '#fcffa4',
 0.0005: '#fbbe23',
 0.0002: '#ea632a',
 0.0001: '#bc3754',
 5e-05: '#7f1e6c',
 2e-05: '#2b0b57',
 1e-05: '#000004'}


In [168]:
map = MapLayout(data = [styled_crustal, polygon])

html = HTML("<b>Section Detail</b><br/><p>hover over fault sections for more details.</p>", style={"background":"white", "border-radius":"5px"})
html.add_class("mapHtmlWidget")
def on_hover(msg):
    properties = msg["properties"]
    value = f"<b>{properties['FaultName']}</b>"
    value += "<br />"
    value += f"Dip: {properties['DipDeg']}</br>"
    value += f"Rake: {properties['Rake']}</br>"
    value += f"Lower depth: {round(properties['LowDepth'],3)}</br>"    
    value += f"Participation rate: {properties['annual_rate']:.2E}</br>" 
    html.value = value

map.map.on_hover(on_hover)
map.map.hover_style = {"fillColor":"red", "fillOpacity":"0.6", "color":"green"}

map.add(html)
map.add(slider(map.map))
map.add(HomeWidget(map.map), "top-left")
map.add(FullScreenWidget(), "top-left")
map.add(legend("Rupture Rate/yr", color_map), "top-left")
map.render()


GridBox(children=(MapWidget(data=[{'type': 'FeatureCollection', 'features': [{'id': '216', 'type': 'Feature', …